In [ ]:
import os
import sunpy.map
import numpy as np
import pandas as pd
import plotly.io as pio
from datetime import datetime
from matplotlib import colormaps, pyplot as plt

import prepare_data
import detect
import plot_detection
from settings import *


pio.renderers.default = 'vscode'
%matplotlib inline

# Data Inspection

## Prepare Data

Extract Data from File System

In [ ]:
# Extract He I observation datetimes from FITS files
HE_DATE_LIST = prepare_data.get_fits_date_list(
    DATE_RANGE, ALL_HE_DIR, SELECT_HE_DIR
)

# Extract magnetogram datetimes from 6302l FITS files
MAG_DATE_LIST = prepare_data.get_fits_date_list(
    DATE_RANGE, ALL_MAG_DIR, SELECT_MAG_DIR
)

# Extract EUV datetimes from FITS files
EUV_DATE_LIST = prepare_data.get_fits_date_list(
    DATE_RANGE, ALL_EUV_DIR, SELECT_EUV_DIR
)

date_strs = [HE_DATE_LIST[0], HE_DATE_LIST[-1]]
file_date_str = f'{date_strs[0]}_{date_strs[-1]}'

num_maps = len(HE_DATE_LIST)
datetimes = [datetime.strptime(date_str, DICT_DATE_STR_FORMAT)
             for date_str in date_strs]
title_date_strs = [datetime.strftime(d, '%m/%d/%Y') for d in datetimes]
DATE_RANGE_SUPTITLE = (f'{num_maps} Maps Evaluated from '
                       + f'{title_date_strs[0]} to {title_date_strs[-1]}')

Rename Data

In [ ]:
# Delete all gzipped files after renaming
remove_gzip = True

# Rename all He, magnetogram, and EUV FITS files to include
# observation date in title
prepare_data.rename_dir(ALL_HE_DIR, remove_gzip)
prepare_data.rename_dir(ALL_MAG_DIR, remove_gzip)
prepare_data.rename_dir(ALL_EUV_DIR, remove_gzip)

## EUV Download

Automatic

In [ ]:
from datetime import datetime, timedelta
DICT_DATE_STR_FORMAT = '%Y_%m_%d__%H_%M'

# Dates for which to download EUV observations
download_euv_dates = HE_DATE_LIST

# Identify missing dates
available_datetimes = [datetime.strptime(date_str, DICT_DATE_STR_FORMAT).date()
                       for date_str in HE_DATE_LIST]
days_in_period = (available_datetimes[-1] - available_datetimes[0]).days
all_period_dates = set(available_datetimes[0] + timedelta(num_days)
                       for num_days in range(days_in_period))
missing_dates = all_period_dates - set(available_datetimes)

# Missing datetime strings shifted to noon
missing_date_str_list = [
    datetime.strftime(missing_date, DICT_DATE_STR_FORMAT)
    for missing_date in missing_dates
]
missing_date_str_list = [
    missing_date_str[:12] + '16_00'
    for missing_date_str in missing_date_str_list
]

# Add missing datetimes
download_euv_dates.extend(missing_date_str_list)
download_euv_dates.sort()

In [ ]:
download_euv_dates = download_euv_dates[97:98]

In [ ]:
prepare_data.download_euv(download_euv_dates, EUV_DATE_LIST,
                          download_dir=ALL_EUV_DIR, hr_window=3)

Manual

In [ ]:
import astropy.units as u
from datetime import datetime, timedelta
from sunpy.net import Fido, attrs as a


date_str = '2012_05_12__18_28'
hr_window = 1

center_date = datetime.strptime(date_str, DICT_DATE_STR_FORMAT)
        
min_date = center_date - timedelta(hours=hr_window)
max_date = center_date + timedelta(hours=hr_window)

result = Fido.search(
    a.Time(min_date, max_date),
    a.Instrument.aia, a.Wavelength(193*u.angstrom),
    a.Sample(30*u.minute), 
)
result

In [ ]:
row_num = 2
download_dir = ALL_EUV_DIR

Fido.fetch(result[:, row_num], path=download_dir + '{file}')

## Available Data

In [ ]:
print('Available Datetimes for He I Observations:')
prepare_data.display_dates(HE_DATE_LIST)

In [ ]:
print('Available Datetimes for Magnetograms:')
prepare_data.display_dates(MAG_DATE_LIST)

In [ ]:
print('Available Datetimes for EUV Observations:')
prepare_data.display_dates(EUV_DATE_LIST)

Magnetogram & EUV Comparison

In [ ]:
smooth_size_percent = 10

for he_date_str in HE_DATE_LIST[:2]:

    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    euv_date_str = prepare_data.get_nearest_date_str(
        date_str_list=EUV_DATE_LIST, selected_date_str=he_date_str
    )

    he_fits_file = prepare_data.get_fits_path(
        he_date_str, DATE_RANGE, ALL_HE_DIR, SELECT_HE_DIR
    )
    mag_fits_file = prepare_data.get_fits_path(
        mag_date_str, DATE_RANGE, ALL_MAG_DIR, SELECT_MAG_DIR
    )
    euv_fits_file = prepare_data.get_fits_path(
        euv_date_str, DATE_RANGE, ALL_EUV_DIR, SELECT_EUV_DIR
    )
    he_map = prepare_data.get_solis_sunpy_map(he_fits_file)
    mag_map = prepare_data.get_solis_sunpy_map(mag_fits_file)
    euv_map = sunpy.map.Map(euv_fits_file)

    # Process magnetogram
    smoothed_mag_map = prepare_data.get_smoothed_map(mag_map, smooth_size_percent)

    fig = plt.figure(figsize=(18, 5))

    ax = fig.add_subplot(131, projection=he_map)
    he_map.plot(axes=ax, vmin=-100, vmax=100, title=he_date_str)

    ax = fig.add_subplot(132, projection=mag_map)
    mag_map.plot(axes=ax, vmin=-50, vmax=50, title=mag_date_str)
    plot_detection.plot_map_contours(ax, smoothed_mag_map)
    
    ax = fig.add_subplot(133, projection=euv_map)
    euv_map.plot(axes=ax, title=euv_date_str)

# Data Products

In [ ]:
he_date_str = HE_DATE_LIST[0]
percent_of_peak_list = [80,80, 90, 100,100]
morph_radius_list = [18,20, 16, 16,20]


# Extract pre-processed map
he_fits_file = prepare_data.get_fits_path(
    he_date_str, DATE_RANGE, ALL_HE_DIR, SELECT_HE_DIR
) 
# raw_he = prepare_data.get_image_from_fits(he_fits_file)
# pre_processed_map_data = detect.pre_process_v0_4(raw_he)
he_map = prepare_data.get_solis_sunpy_map(he_fits_file)
pre_processed_map = detect.pre_process_vX(he_map)
pre_processed_map_data = np.flipud(pre_processed_map.data)

out = detect.get_ensemble_v0_3(
    pre_processed_map_data, percent_of_peak_list, morph_radius_list
)
ensemble_map, map_data_by_ch, confidence_list, gradient_medians = out
plot_detection.plot_ensemble(
    pre_processed_map_data, ensemble_map, map_data_by_ch,
    confidence_list, gradient_medians
)

## Pre-Processed Files

### Pre-Processed Maps

In [ ]:
overwrite = False

if not os.path.isdir(PREPROCESS_SAVE_DIR):
    os.makedirs(PREPROCESS_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    euv_date_str = prepare_data.get_nearest_date_str(
        EUV_DATE_LIST, selected_date_str=he_date_str
    )
    
    # Optionally overwrite existing files
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    if os.path.isfile(pre_process_file) and not overwrite:
        print((f'He {he_date_str} pre-processed map already exists.'))
        continue
    
    # he_fits_file = prepare_data.get_fits_path(
    #     he_date_str, DATE_RANGE, ALL_HE_DIR, SELECT_HE_DIR
    # ) 
    he_fits_file = f'{RATIO_SAVE_DIR}He{he_date_str}_EUV{euv_date_str}.fits'

    raw_he = prepare_data.get_image_from_fits(he_fits_file)
    
    # pre_processed_map = detect.pre_process_v0_1(raw_he)[0]
    pre_processed_map = detect.pre_process_v0_4(raw_he)
    
    save_list = [he_date_str, pre_processed_map]
    np.save(pre_process_file, np.array(save_list, dtype=object), 
            allow_pickle=True)
    print(f'{he_date_str} Pre-Processed Map Saved')

v0.6+

In [ ]:
overwrite = False

if not os.path.isdir(PREPROCESS_MAP_SAVE_DIR):
    os.makedirs(PREPROCESS_MAP_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    if os.path.isfile(pre_process_file) and not overwrite:
        print((f'He {he_date_str} pre-processed map already exists.'))
        continue
    
    he_fits_file = prepare_data.get_fits_path(
        he_date_str, DATE_RANGE, ALL_HE_DIR, SELECT_HE_DIR
    )
    he_map = prepare_data.get_solis_sunpy_map(he_fits_file)
    
    pre_processed_map = detect.pre_process_vY(he_map)
    
    pre_processed_map.save(pre_process_file, overwrite=overwrite)
    print(f'{he_date_str} Pre-Processed Map Saved')

### Reprojected Magnetograms

Differential rotation

In [ ]:
overwrite = True
smooth_size_percent = 10

if not os.path.isdir(ROTATED_MAG_SAVE_DIR):
    os.makedirs(ROTATED_MAG_SAVE_DIR)

for he_date_str in HE_DATE_LIST[1:]:

    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    fits_file_name = f'{ROTATED_MAG_SAVE_DIR}Mag{mag_date_str}_He{he_date_str}'
    reprojected_fits_file = f'{fits_file_name}.fits'
    reprojected_smooth_fits_file = f'{fits_file_name}_smooth.fits'
    
    # Optionally overwrite existing files
    if (os.path.isfile(reprojected_fits_file) or \
        os.path.isfile(reprojected_smooth_fits_file)) and not overwrite:
        print((f'{mag_date_str} magnetogram reprojected '
                + f'to {he_date_str} already exists.'))
        continue
    
    # Extract He I observation
    he_map = prepare_data.get_solis_sunpy_map(ALL_HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    # Extract Magnetogram observation
    mag_map = prepare_data.get_solis_sunpy_map(ALL_MAG_DIR + mag_date_str + '.fts')

    # Process magnetogram
    reprojected_mag_map = prepare_data.diff_rotate(
        input_map=mag_map, target_map=he_map
    )
    
    smoothed_map = prepare_data.get_smoothed_map(mag_map, smooth_size_percent)
    reprojected_smooth_map = prepare_data.diff_rotate(
        input_map=smoothed_map, target_map=he_map
    )
    
    # Save to FITS files
    reprojected_mag_map.save(reprojected_fits_file, overwrite=overwrite)
    reprojected_smooth_map.save(f'{fits_file_name}_smooth.fits', overwrite=overwrite)
    print(f'{mag_date_str} magnetogram reprojected to {he_date_str} maps saved.')

Heliographic reprojection

In [ ]:
overwrite = True
smooth_size_percent = 10

if not os.path.isdir(HELIOGRAPH_MAG_SAVE_DIR):
    os.makedirs(HELIOGRAPH_MAG_SAVE_DIR)

for he_date_str in HE_DATE_LIST:

    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    fits_file_name = f'{HELIOGRAPH_MAG_SAVE_DIR}Mag{mag_date_str}_He{he_date_str}'
    reprojected_fits_file = f'{fits_file_name}.fits'
    reprojected_smooth_fits_file = f'{fits_file_name}_smooth.fits'
    
    # Optionally overwrite existing files
    if (os.path.isfile(reprojected_fits_file) or \
        os.path.isfile(reprojected_smooth_fits_file)) and not overwrite:
        print((f'{mag_date_str} magnetogram reprojected '
                + f'to {he_date_str} already exists.'))
        continue
    
    # Extract observations
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)
    
    mag_fits_file = prepare_data.get_fits_path(
        mag_date_str, DATE_RANGE, ALL_MAG_DIR, SELECT_MAG_DIR
    )
    mag_map = prepare_data.get_solis_sunpy_map(mag_fits_file)

    # Process magnetogram
    hg_mag_map = detect.reproject_to_cea(mag_map)
    reprojected_mag_map = hg_mag_map.reproject_to(
        pre_processed_map.wcs, algorithm='adaptive'
    )
    
    smoothed_map = prepare_data.get_smoothed_map(mag_map, smooth_size_percent)
    hg_smoothed_map = detect.reproject_to_cea(smoothed_map)
    reprojected_smooth_map = hg_smoothed_map.reproject_to(
        pre_processed_map.wcs, algorithm='adaptive'
    )
    
    # Save to FITS files
    reprojected_mag_map.save(reprojected_fits_file, overwrite=overwrite)
    reprojected_smooth_map.save(reprojected_smooth_fits_file, overwrite=overwrite)
    print(f'{mag_date_str} magnetogram reprojected to {he_date_str} map saved.')

### Create He/EUV Ratios

In [ ]:
overwrite = False


if not os.path.isdir(RATIO_SAVE_DIR):
    os.makedirs(RATIO_SAVE_DIR)

for he_date_str in HE_DATE_LIST:

    euv_date_str = prepare_data.get_nearest_date_str(
        EUV_DATE_LIST, selected_date_str=he_date_str
    )
    
    # Optionally overwrite existing files
    ratio_fits_file = f'{RATIO_SAVE_DIR}He{he_date_str}_EUV{euv_date_str}.fits'
    if os.path.isfile(ratio_fits_file):
        if overwrite:
            os.remove(ratio_fits_file)
        else:
            print((f'{he_date_str} to {euv_date_str} ratio already exists.'))
            continue
    
    # Extract He I observation
    he_fits_file = prepare_data.get_fits_path(
        he_date_str, DATE_RANGE, ALL_HE_DIR, SELECT_HE_DIR
    )
    he_map = prepare_data.get_solis_sunpy_map(he_fits_file)
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    # Remove error causing keywords which have invalid ascii content
    he_map.meta.pop('history')
    he_map.meta.pop('comment')
    
    # Extract and reproject EUV observation
    euv_fits_file = prepare_data.get_fits_path(
        euv_date_str, DATE_RANGE, ALL_EUV_DIR, SELECT_EUV_DIR
    )
    euv_map = sunpy.map.Map(euv_fits_file)
    reprojected_euv_map = prepare_data.diff_rotate(
        input_map=euv_map, target_map=he_map
    )
    
    # Pre-process He I data via background removal and upper cutoff
    he_map_data = np.where(he_map.data == he_map.data[0,0],
                           np.nan, he_map.data)
    he_map_data = np.where(he_map_data >= np.percentile(he_map_data, 99.9),
                           np.nan, he_map_data)
    
    ratio_data = np.divide(he_map_data, (reprojected_euv_map.data)**0.5)
    ratio_map = sunpy.map.Map(ratio_data, he_map.meta)
    
    # Save to FITS files
    ratio_map.save(ratio_fits_file)
    print(f'He{he_date_str} to EUV{euv_date_str} ratio saved.')

## Detection Files

### Single Masks

In [ ]:
overwrite = False

percent_of_peak = 80
morph_radius = 18


if not os.path.isdir(DETECTION_SAVE_DIR):
    os.makedirs(DETECTION_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    mask_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    if os.path.isfile(mask_file) and not overwrite:
        print((f'He {he_date_str} single mask already exists.'))
        continue
    
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]

    ch_mask = detect.get_ch_mask(
        pre_processed_map, percent_of_peak, morph_radius
    )
    
    save_list = [he_date_str, ch_mask]
    np.save(mask_file, np.array(save_list, dtype=object), allow_pickle=True)
    print(f'{he_date_str} Single Mask Saved')

### Ensemble Maps

In [ ]:
overwrite = True

# percent_of_peak_list = [80,80, 90, 100,100]
# morph_radius_list = [18,20, 16, 16,20]
percent_of_peak_list = [100,100, 110, 120,120]
morph_radius_list = [18,20, 16, 16,20]


if not os.path.isdir(DETECTION_SAVE_DIR):
    os.makedirs(DETECTION_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    if os.path.isfile(ensemble_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]

    ensemble_map = detect.get_ensemble_v0_3(
        pre_processed_map, percent_of_peak_list, morph_radius_list
    )[0]
    
    save_list = [he_date_str, ensemble_map]
    np.save(ensemble_file, np.array(save_list, dtype=object), allow_pickle=True)
    print(f'{he_date_str} Ensemble Map Saved')

v0.5

In [ ]:
overwrite = False

percent_of_peak_list = [80,80, 90, 100,100]
morph_radius_list = [18,20, 16, 16,20]
unipolarity_threshold = 0.5


if not os.path.isdir(DETECTION_SAVE_DIR):
    os.makedirs(DETECTION_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    if os.path.isfile(ensemble_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    # Extract He I observation
    he_file = f'{ALL_HE_DIR}{he_date_str}.fts'
    he_map = prepare_data.get_solis_sunpy_map(he_file)

    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map_data = np.load(pre_process_file, allow_pickle=True)[-1]
    pre_processed_map = sunpy.map.Map(
        np.flipud(pre_processed_map_data), he_map.meta
    )

    # Extract saved processed magnetograms
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    reprojected_fits_file = (f'{ROTATED_MAG_SAVE_DIR}'
                            f'Mag{mag_date_str}_He{he_date_str}.fits')
    reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)

    ensemble_map_data = detect.get_ensemble_v0_5(
        pre_processed_map, reprojected_mag_map,
        percent_of_peak_list, morph_radius_list,
        unipolarity_threshold
    )[0]
    
    save_list = [he_date_str, ensemble_map_data]
    np.save(ensemble_file, np.array(save_list, dtype=object), allow_pickle=True)
    print(f'{he_date_str} Ensemble Map Saved')

vY

In [ ]:
overwrite = False

# percent_of_peak_list = [  62, 68, 73, 80]
# morph_radius_list = [11, 13,  8, 10]
# unipolarity_threshold = 0.01
percent_of_peak_list = [85, 73, 95, 85]
morph_radius_list = [   10, 14, 10, 14]
unipolarity_threshold = 0.5



if not os.path.isdir(DETECTION_MAP_SAVE_DIR):
    os.makedirs(DETECTION_MAP_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    if os.path.isfile(ensemble_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    # Extract pre-processed map
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)

    # Extract saved processed magnetograms
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    reprojected_fits_file = (f'{HELIOGRAPH_MAG_SAVE_DIR}'
                            f'Mag{mag_date_str}_He{he_date_str}.fits')
    reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)

    ensemble_map_data = detect.get_ensemble_vY(
        pre_processed_map, reprojected_mag_map,
        percent_of_peak_list, morph_radius_list,
        unipolarity_threshold
    )[0]
    ensemble_map = sunpy.map.Map(
        np.flipud(ensemble_map_data), pre_processed_map.meta
    )
    
    ensemble_map.save(ensemble_file, overwrite=overwrite)
    print(f'{he_date_str} Ensemble Map Saved')

## Detection Map Images

### He & Single Mask

In [ ]:
out_dir = DETECTION_IMAGE_DIR + 'Preprocess_Single_Comparison/'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST:
    
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]
    
    he_fits_file = prepare_data.get_fits_path(
        he_date_str, DATE_RANGE, ALL_HE_DIR, SELECT_HE_DIR
    ) 
    raw_he = prepare_data.get_image_from_fits(he_fits_file)
    he = np.where(raw_he == raw_he[0,0], np.NaN, raw_he)
    
    mask_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ch_mask.npy'
    single_mask = np.load(mask_file, allow_pickle=True)[-1]

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
    ax = axes.ravel()

    ax[0].set_title(he_date_str, fontsize=24)
    ax[0].imshow(pre_processed_map, cmap=plt.cm.gray)
    
    ax[1].imshow(he, cmap=plt.cm.afmhot, vmin=-100, vmax=100)
    ax[1].contour(single_mask, linewidths=2, cmap=plt.cm.gray)

    plt.savefig(f'{out_dir}{he_date_str}.jpg')
    plt.close(fig)
    print(f'{he_date_str} map saved.')

### He & Ensemble Map

In [ ]:
out_dir = DETECTION_IMAGE_DIR + 'Preprocess_Comparison/'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST:
    
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]
    
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map = np.load(ensemble_file, allow_pickle=True)[-1]

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
    ax = axes.ravel()

    ax[0].set_title(he_date_str, fontsize=24)
    ax[0].imshow(pre_processed_map, cmap=plt.cm.gray)
    
    ax[1].imshow(ensemble_map, cmap=plt.cm.magma)

    plt.savefig(f'{out_dir}{he_date_str}.jpg')
    plt.close(fig)
    print(f'{he_date_str} map saved.')

vY

In [ ]:
out_dir = DETECTION_IMAGE_DIR + 'Preprocess_Comparison/'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST:
    
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)
    
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    ensemble_map = sunpy.map.Map(ensemble_file)

    fig = plt.figure(figsize=(18, 5))
    ax = fig.add_subplot(1, 5, (1,2), projection=pre_processed_map)
    pre_processed_map.plot(axes=ax, title=he_date_str)
    
    # Plot ensemble map with overlayed neutral lines
    ax = fig.add_subplot(1, 5, (3,5), projection=ensemble_map)
    ensemble_map.plot(axes=ax, title='', cmap='magma')

    plt.savefig(f'{out_dir}{he_date_str}.jpg')
    plt.close(fig)
    print(f'{he_date_str} map saved.')

### He, Ensemble, Mag, & EUV

In [ ]:
overwrite = True
out_dir = DETECTION_IMAGE_DIR + 'Full_Comparison/'
smooth_size_percent = 10

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} full comparison already exists.'))
        continue

    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    # Extract He I observation
    he_map = prepare_data.get_solis_sunpy_map(ALL_HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    mag_map = prepare_data.get_solis_sunpy_map(ALL_MAG_DIR + mag_date_str + '.fts')
    euv_map = sunpy.map.Map(ALL_EUV_DIR + euv_date_str + '.fts')

    # Process magnetogram
    smoothed_mag_map = prepare_data.get_smoothed_map(mag_map, smooth_size_percent)
    
    # Extract saved ensemble map array and convert to Sunpy map
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map_data = np.load(ensemble_file, allow_pickle=True)[-1]
    ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), he_map.meta)
    ensemble_map.plot_settings['cmap'] = colormaps['magma']
    
    fig = plt.figure(figsize=(24, 5))

    ax = fig.add_subplot(141, projection=he_map)
    he_map.plot(axes=ax, vmin=-100, vmax=100, title=he_date_str)

    ax = fig.add_subplot(142, projection=he_map)
    ensemble_map.plot(axes=ax, title='')
    
    ax = fig.add_subplot(143, projection=mag_map)
    mag_map.plot(axes=ax, vmin=-50, vmax=50, title=mag_date_str)
    plot_detection.plot_map_contours(ax, smoothed_mag_map)
    
    ax = fig.add_subplot(144, projection=euv_map)
    euv_map.plot(axes=ax, title=euv_date_str)
    
    plt.savefig(comparison_img_file)
    plt.close(fig)

    print(f'{euv_date_str} map comparison saved.')

#### He, Ensemble, Mag with Neutral Lines

In [ ]:
out_dir = DETECTION_IMAGE_DIR + 'Mag_Comparison/'

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST[-10:]:

    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )

    # Extract He I observation
    he_map = prepare_data.get_solis_sunpy_map(ALL_HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue

    # Extract saved processed magnetograms
    mag_fits_name = f'{ROTATED_MAG_SAVE_DIR}Mag{mag_date_str}_He{he_date_str}'
    reprojected_fits_file = f'{mag_fits_name}.fits'
    reprojected_smooth_fits_file = f'{mag_fits_name}_smooth.fits'
    reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)
    reprojected_smooth_map = sunpy.map.Map(reprojected_smooth_fits_file)
    # reprojected_smooth_map = sunpy.map.Map(np.fliplr(reprojected_smooth_map.data), reprojected_smooth_map.meta)

    # Extract saved ensemble map array and convert to Sunpy map
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map_data = np.load(ensemble_file, allow_pickle=True)[-1]
    ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), he_map.meta)
    ensemble_map.plot_settings['cmap'] = colormaps['magma']

    fig = plt.figure(figsize=(18, 5))

    ax = fig.add_subplot(131, projection=he_map)
    he_map.plot(axes=ax, vmin=-100, vmax=100, title=he_date_str)

    ax = fig.add_subplot(132, projection=he_map)
    ensemble_map.plot(axes=ax, title='')
    plot_detection.plot_map_contours(ax, reprojected_smooth_map)

    ax = fig.add_subplot(133, projection=he_map)
    reprojected_mag_map.plot(axes=ax, vmin=-50, vmax=50, title=mag_date_str)
    plot_detection.plot_map_contours(ax, reprojected_smooth_map)

    plt.savefig(f'{out_dir}He{he_date_str}.jpg')
    plt.close(fig)

    print(f'{he_date_str} map comparison saved')

#### He, Ensemble, Neutral Lines, & EUV

In [ ]:
overwrite = False
out_dir = DETECTION_IMAGE_DIR + 'EUV_Comparison/'

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} comparison already exists.'))
        continue
    
    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    fig = plt.figure(figsize=(18, 5))
    plot_detection.plot_he_neutral_lines_euv_comparison(
        fig, he_date_str, mag_date_str, euv_date_str,
        ROTATED_MAG_SAVE_DIR
    )
    
    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{euv_date_str} map comparison saved.')

v0.6+

In [ ]:
from plot_detection import plot_map_contours


def plot_he_neutral_euv_comparison_v0_6(fig, he_date_str, mag_date_str,
                                        euv_date_str, nrows=1):
    """Plot a 3 panel comparison of a saturated He I observation, ensemble map
    with neutral lines, and an EUV observation.
    """
    # Extract He I observation
    he_map = prepare_data.get_solis_sunpy_map(ALL_HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
    
    # Extract saved ensemble map array and convert to Sunpy map
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    ensemble_map = sunpy.map.Map(ensemble_file)
    ensemble_map.plot_settings['cmap'] = colormaps['magma']

    # Extract saved processed magnetogram
    reprojected_smooth_file = (f'{HELIOGRAPH_MAG_SAVE_DIR}Mag{mag_date_str}'
                               + f'_He{he_date_str}_smooth.fits')
    reprojected_smooth_map = sunpy.map.Map(reprojected_smooth_file)
    
    euv_map = sunpy.map.Map(ALL_EUV_DIR + euv_date_str + '.fts')
    
    # Plot He observation
    ax = fig.add_subplot(nrows, 7, (1,2), projection=he_map)
    he_map.plot(axes=ax, vmin=-100, vmax=100, title=he_date_str)
    
    # Plot ensemble map with overlayed neutral lines
    ax = fig.add_subplot(nrows, 7, (3,5), projection=ensemble_map)
    ensemble_map.plot(axes=ax, title=' ')
    plot_map_contours(ax, reprojected_smooth_map)
    
    # Plot EUV observation
    ax = fig.add_subplot(nrows, 7, (6,7), projection=euv_map)
    euv_map.plot(axes=ax, title=euv_date_str)

In [ ]:
overwrite = False
out_dir = DETECTION_IMAGE_DIR + 'EUV_Comparison/'

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} comparison already exists.'))
        continue
    
    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    fig = plt.figure(figsize=(22, 5))
    plot_he_neutral_euv_comparison_v0_6(
        fig, he_date_str, mag_date_str, euv_date_str
    )
    
    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{euv_date_str} map comparison saved.')

Version Comparison

In [ ]:
overwrite = True
# out_dir = (DETECTION_IMAGE_DIR + '_Version_Comparison'
#            + DATE_DIR + 'v0_2_v0_4_Unipolar/')
out_dir = (DETECTION_IMAGE_DIR + '_Version_Comparison'
           + DATE_DIR + 'v0_5_v0_6/')

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

# comparison_version_dir = DETECT_DIR + 'v0_1/' + 'Saved_npy_Files/'
comparison_version_dir = DETECT_DIR + 'v0_5/' + 'Saved_npy_Files/'

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} comparison already exists.'))
        continue
    
    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    # Extract He I observation
    he_map = prepare_data.get_solis_sunpy_map(ALL_HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
    
    # Extract comparison saved ensemble map array and convert to Sunpy map
    comparison_file = f'{comparison_version_dir}{he_date_str}_ensemble_map.npy'
    comparison_map_data = np.load(comparison_file, allow_pickle=True)[-1]
    comparison_map_data = np.where(np.isnan(ensemble_map_data), np.nan, comparison_map_data)
    comparison_map = sunpy.map.Map(np.flipud(comparison_map_data), he_map.meta)
    
    # Extract saved ensemble map array and convert to Sunpy map
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map_data = np.load(ensemble_file, allow_pickle=True)[-1]
    ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), he_map.meta)
    ensemble_map.plot_settings['cmap'] = colormaps['magma']

    
    # Extract saved processed magnetogram
    reprojected_smooth_file = (f'{ROTATED_MAG_SAVE_DIR}Mag{mag_date_str}'
                               + f'_He{he_date_str}_smooth.fits')
    reprojected_smooth_map = sunpy.map.Map(reprojected_smooth_file)
    
    euv_map = sunpy.map.Map(ALL_EUV_DIR + euv_date_str + '.fts')
    
    fig = plt.figure(figsize=(12, 11))
    
    # Plot He observation
    ax = fig.add_subplot(2, 2, 1, projection=he_map)
    he_map.plot(axes=ax, vmin=-100, vmax=100, title=he_date_str)
    
    # Plot EUV observation
    ax = fig.add_subplot(2, 2, 2, projection=euv_map)
    euv_map.plot(axes=ax, title=euv_date_str)
    
    # Plot ensemble map with overlayed neutral lines
    ax = fig.add_subplot(2, 2, 3, projection=he_map)
    comparison_map.plot(axes=ax, title='v0.5')
    plot_detection.plot_map_contours(ax, reprojected_smooth_map)
    
    # Plot ensemble map with overlayed neutral lines
    ax = fig.add_subplot(2, 2, 4, projection=he_map)
    ensemble_map.plot(axes=ax, title='v0.6')
    plot_detection.plot_map_contours(ax, reprojected_smooth_map)

    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{euv_date_str} map comparison saved.')

### EUV Ratio Map Comparison

In [ ]:
overwrite = False
out_dir = out_dir + 'Ratio_Comparison/' + DATE_DIR

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}He{he_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'He {he_date_str} ratio comparison already exists.'))
        continue

    euv_date_str = prepare_data.get_nearest_date_str(
        EUV_DATE_LIST, selected_date_str=he_date_str
    )
    
    # Extract observations and ratio map
    he_fits_file = prepare_data.get_fits_path(
        he_date_str, DATE_RANGE, ALL_HE_DIR, SELECT_HE_DIR
    )
    he_map = prepare_data.get_solis_sunpy_map(he_fits_file)
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    euv_fits_file = prepare_data.get_fits_path(
        euv_date_str, DATE_RANGE, ALL_EUV_DIR, SELECT_EUV_DIR
    )
    euv_map = sunpy.map.Map(euv_fits_file)
    
    ratio_fits_file = f'{RATIO_SAVE_DIR}He{he_date_str}_EUV{euv_date_str}.fits'
    ratio_map = sunpy.map.Map(ratio_fits_file)
    
    # Create panel plot
    fig = plt.figure(figsize=(18, 5))

    ax = fig.add_subplot(131, projection=he_map)
    he_map.plot(axes=ax, title=he_date_str, vmin=-100, vmax=100)

    ax = fig.add_subplot(132, projection=euv_map)
    euv_map.plot(axes=ax, title=euv_date_str)

    ax = fig.add_subplot(133, projection=he_map)
    ratio_map.plot(axes=ax, cmap='jet', vmin=-1, vmax=6)
    
    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'He {he_date_str} map comparison saved.')

## Outcome Images

### Time Series Outcomes

#### Pre-Process

Histogram Moments vs Time

In [ ]:
import pandas as pd
from datetime import datetime

hist_stat_list = []

for he_date_str in HE_DATE_LIST:
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map_data = np.load(pre_process_file, allow_pickle=True)[-1]
    
    peak_counts_val = detect.get_peak_counts_loc(
        pre_processed_map_data, bins_as_percent=False
    )
    hist_stat_list.append(
        [peak_counts_val, np.nanstd(pre_processed_map_data)]
    )

# Convert to dataframes
datetime_list = [datetime.strptime(he_date_str, DICT_DATE_STR_FORMAT)
                 for he_date_str in HE_DATE_LIST]
hist_df = pd.DataFrame(
    hist_stat_list, columns=['Peak', 'StDev'],
    index=datetime_list
)

In [ ]:
overwrite = True
out_dir = DETECTION_IMAGE_DIR + 'Histogram_Moments/'
cmap = 'plasma'
ylabel = 'Histogram Moments'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    img_file = f'{out_dir}{he_date_str}.jpg'
    if os.path.isfile(img_file) and not overwrite:
        print((f'He {he_date_str} map already exists.'))
        continue
    
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]
    
    hist, edges = detect.get_hist(pre_processed_map,
                                         bins_as_percent=False)
    
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map = np.load(ensemble_file, allow_pickle=True)[-1]
    
    fig = plt.figure(figsize=(18, 10))
    
    ax = fig.add_subplot(231)
    ax.set_title(he_date_str)
    ax.imshow(pre_processed_map, cmap=plt.cm.gray)
    
    ax = fig.add_subplot(232)
    ax.set_title('Semilog Histogram')
    ax.semilogy(edges[1:], hist)
    if 'Rescale' in DETECTION_VERSION_DIR:
        ax.set_xlim([-1.3, 1.1])
        ax.set_ylim([1E2, 5E4])
    else:
        ax.set_xlim([-110, 110])
        ax.set_ylim([1E1, 5E4])
    
    ax = fig.add_subplot(233)
    ax.imshow(ensemble_map, cmap=plt.cm.magma)
    
    ax = fig.add_subplot(2, 3, (4, 6))
    datetimes = hist_df.index
    ax.plot(hist_df['StDev'], label='Standard Deviation', linewidth=3)
    ax.plot(hist_df['Peak'], label='Mode', linewidth=3)
    
    # Vertical line for datetime indicator
    vline_datetime = datetime.strptime(he_date_str, DICT_DATE_STR_FORMAT)
    min_moment = min(hist_df.min())
    max_moment = max(hist_df.max())
    ax.vlines(x=[vline_datetime, vline_datetime], ymax=2*max_moment, ymin=0,
              colors='k', linestyles='dashed')
    
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    ax.set_ylabel(ylabel)
    
    ax.set_xlim([datetimes[0], datetimes[-1]])
    if 'Rescale' in DETECTION_VERSION_DIR:
        ax.set_ylim([0.4, 0.8])
    else:
        ax.set_ylim([0.9*min_moment, 1.1*max_moment])
    
    ax.legend(reverse=True)

    plt.savefig(img_file)
    plt.close(fig)
    print(f'{he_date_str} map saved.')

Pre-Process Outcomes vs Time

In [ ]:
percent_of_peak_list = [80, 90, 100, 110]
num_ch_df, area_percent_df, area_df, px_percent_df = detect.get_thresh_outcome_time_series_dfs(
    HE_DATE_LIST, percent_of_peak_list, ALL_HE_DIR, PREPROCESS_SAVE_DIR
)

In [ ]:
overwrite = True
out_dir = DETECTION_IMAGE_DIR + 'Thresh_Area_Percentage/'
outcome_df = area_percent_df
cmap = 'plasma'
ylabel = 'Detected Area Percentage (%)'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST[24:25]:
    
    # Optionally overwrite existing files
    img_file = f'{out_dir}{he_date_str}.jpg'
    if os.path.isfile(img_file) and not overwrite:
        print((f'He {he_date_str} map already exists.'))
        continue
    
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]
    
    hist, edges = detect.get_hist(pre_processed_map,
                                         bins_as_percent=False)
    
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map = np.load(ensemble_file, allow_pickle=True)[-1]
    
    fig = plt.figure(figsize=(18, 10))
    
    ax = fig.add_subplot(231)
    ax.set_title(he_date_str)
    ax.imshow(pre_processed_map, cmap=plt.cm.gray)
    
    ax = fig.add_subplot(232)
    ax.set_title('Semilog Histogram')
    ax.semilogy(edges[1:], hist)
    if 'Rescale/' in DETECTION_VERSION_DIR:
        ax.set_ylim([1E2, 5E4])
    else:
        ax.set_xlim([-110, 110])
        ax.set_ylim([1E1, 5E4])
    
    ax = fig.add_subplot(233)
    ax.imshow(ensemble_map, cmap=plt.cm.magma)
    
    ax = fig.add_subplot(2, 3, (4, 6))    
    plot_detection.plot_thresh_outcome_vs_time(
        ax, outcome_df, he_date_str, cmap, ylabel)

    plt.savefig(img_file)
    plt.close(fig)
    print(f'{he_date_str} map saved.')

#### Detection

Single Mask vs Time

In [ ]:
from matplotlib import colormaps


def get_outcome_series(he_date_str_list, he_dir, DETECTION_SAVE_DIR):
    """Retrieve series with single mask outcomes over time.
    
    Args
        he_date_str_list: list of date strings for ensemble maps
        he_dir: path to directory with saved He I observations
        DETECTION_SAVE_DIR: path to directory with saved single masks
    Returns
        Dataframes of outcomes by confidence level over time.
    """
    # List for outcomes at varied confidence levels and datetimes
    num_ch_list = []
    area_percent_list = []
    area_list = []
    px_percent_list = []

    for he_date_str in he_date_str_list:
        
        he_file = f'{he_dir}{he_date_str}.fts'
        he_map = prepare_data.get_solis_sunpy_map(he_file)
        if not he_map:
            print(f'{he_date_str} He I observation extraction failed.')
            continue
        
        # Extract saved single mask
        mask_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ch_mask.npy'
        mask_data = np.load(mask_file, allow_pickle=True)[-1]
        mask_map = sunpy.map.Map(np.flipud(mask_data), he_map.meta)
        
        # Lists of CH outcomes
        num_ch_list.append(detect.get_num_ch(mask_data))
        area_tuple = detect.get_open_area(mask_map, confidence_level=0)
        area_percent_list.append(area_tuple[0])
        area_list.append(area_tuple[1])
        px_percent_list.append(detect.get_px_percent(mask_data))
    
    # Convert to dataframes
    datetime_list = [datetime.strptime(he_date_str, DICT_DATE_STR_FORMAT)
                    for he_date_str in he_date_str_list]
    num_ch_series = pd.Series(
        num_ch_list, index=datetime_list
    )
    area_percent_series = pd.Series(
        area_percent_list, index=datetime_list
    )
    area_series = pd.Series(
        area_list, index=datetime_list
    )
    px_percent_series = pd.Series(
        px_percent_list, index=datetime_list
    )
    return num_ch_series, area_percent_series, area_series, px_percent_series


def plot_outcome_vs_time(ax, outcome_series, date_str, cmap, ylabel):
    """Plot outcome stacked plot vs time for ensemble maps.
    
    Args
        ax: matplotlib axes object to plot on
        outcome_series: Pandas dataframe of outcome by confidence level
            over time
        date_str: Date string at which to plot a vertical line
        cmap: Matplotlib colormap name
        ylabel: string for y axis label
    """
    datetimes = outcome_series.index
    ax.fill_between(datetimes, outcome_series, 0, color=colormaps[cmap](0.75))
    
    # Vertical line for datetime indicator
    vline_datetime = datetime.strptime(date_str, DICT_DATE_STR_FORMAT)
    max_outcome = outcome_series.max()
    ax.vlines(x=[vline_datetime, vline_datetime], ymax=1.2*max_outcome, ymin=0,
              colors='k', linestyles='dashed')
    
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    ax.set_ylabel(ylabel)
    ax.set_xlim([datetimes[0], datetimes[-1]])
    ax.set_ylim([0, 1.1*max_outcome])

In [ ]:
num_ch_series, area_percent_series, area_series, px_percent_series = get_outcome_series(
    HE_DATE_LIST, ALL_HE_DIR, DETECTION_SAVE_DIR
)

In [ ]:
overwrite = True
region_num_settings = (
    DETECTION_IMAGE_DIR + 'Single_Region_Number/',
    num_ch_series, 'viridis', 'Detected CH Number'
)
px_percent_settings = (
    DETECTION_IMAGE_DIR + 'Single_EUV_Px_Percentage/',
    px_percent_series, 'plasma', 'Detected Pixel Percentage (%)'
)
area_percent_settings = (
    DETECTION_IMAGE_DIR + 'Single_EUV_Area_Percentage/',
    area_percent_series, 'plasma', 'Detected Area Percentage (%)'
)
area_settings = (
    DETECTION_IMAGE_DIR + 'Single_EUV_Area/',
    area_series, 'plasma', 'Detected Area (Mm^2)'
)
out_dir, outcome_series, cmap, ylabel = area_percent_settings

In [ ]:
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} comparison already exists.'))
        continue

    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    
    # Extract He I observation
    he_map = prepare_data.get_solis_sunpy_map(ALL_HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    # Extract He I observation for mask base and convert to Sunpy map
    he_fits_file = prepare_data.get_fits_path(
        he_date_str, DATE_RANGE, ALL_HE_DIR, SELECT_HE_DIR
    ) 
    raw_he = prepare_data.get_image_from_fits(he_fits_file)
    he_base_data = np.where(raw_he == raw_he[0,0], np.NaN, raw_he)
    he_base_map = sunpy.map.Map(np.flipud(he_base_data), he_map.meta)
    he_base_map.plot_settings['cmap'] = colormaps['afmhot']
    
    # Extract saved single mask array and convert to Sunpy map
    mask_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ch_mask.npy'
    mask_data = np.load(mask_file, allow_pickle=True)[-1]
    mask_map = sunpy.map.Map(np.flipud(mask_data), he_map.meta)
    mask_map.plot_settings['cmap'] = colormaps['gray']
    
    euv_map = sunpy.map.Map(ALL_EUV_DIR + euv_date_str + '.fts')
    
    fig = plt.figure(figsize=(18, 10))
    
    # Plot He observation
    ax = fig.add_subplot(231, projection=he_map)
    he_map.plot(axes=ax, vmin=-100, vmax=100, title=he_date_str)
    
    # Plot ensemble map with overlayed neutral lines
    ax = fig.add_subplot(232, projection=he_map)
    he_base_map.plot(axes=ax, vmin=-100, vmax=100, title=he_date_str)
    for contour in mask_map.contour(0):
        ax.plot_coord(contour, color='black', linewidth=1)
    
    # Plot EUV observation
    ax = fig.add_subplot(233, projection=euv_map)
    euv_map.plot(axes=ax, title=euv_date_str)
    
    ax = fig.add_subplot(2, 3, (4, 6))
    plot_outcome_vs_time(ax, outcome_series, he_date_str, cmap, ylabel)
    
    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{euv_date_str} map comparison saved.')

Ensemble vs Time

In [ ]:
confidence_level_list = [1, 50, 75, 95]
# confidence_level_list = [0, 35, 65, 95]
num_ch_df, area_percent_df, area_df, px_percent_df = detect.get_outcome_time_series_dfs(
    HE_DATE_LIST, confidence_level_list, ALL_HE_DIR, DETECTION_SAVE_DIR
)
area_percent_df[50].median()

In [ ]:
overwrite = True
region_num_settings = (
    DETECTION_IMAGE_DIR + 'Region_Number/',
    num_ch_df, 'viridis', 'Detected CH Number'
)
px_percent_settings = (
    DETECTION_IMAGE_DIR + 'EUV_Px_Percentage/',
    px_percent_df, 'plasma', 'Detected Pixel Percentage (%)'
)
area_percent_settings = (
    DETECTION_IMAGE_DIR + 'EUV_Area_Percentage/',
    area_percent_df, 'plasma', 'Detected Area Percentage (%)'
)
area_settings = (
    DETECTION_IMAGE_DIR + 'EUV_Area/',
    area_df, 'plasma', 'Detected Area (Mm^2)'
)
out_dir, outcome_df, cmap, ylabel = area_percent_settings

In [ ]:
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} comparison already exists.'))
        continue
    
    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    # Extract He I observation
    he_map = prepare_data.get_solis_sunpy_map(ALL_HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    # Extract saved ensemble map array and convert to Sunpy map
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map_data = np.load(ensemble_file, allow_pickle=True)[-1]
    ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), he_map.meta)
    ensemble_map.plot_settings['cmap'] = colormaps['magma']
    
    fig = plt.figure(figsize=(18, 10))
    plot_detection.plot_he_neutral_lines_euv_comparison(
        fig, he_date_str, mag_date_str, euv_date_str,
        ROTATED_MAG_SAVE_DIR, nrows=2
    )
    
    ax = fig.add_subplot(2, 3, (4, 6))
    plot_detection.plot_outcome_vs_time(
        ax, outcome_df, he_date_str, cmap, ylabel)
    
    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{euv_date_str} map comparison saved.')

### Outcome Comparison

Pre-Process Outcomes vs Method

In [ ]:
percent_of_peak_list = [80, 90, 100, 110]
area_percent_df_by_method_list = []
mad_by_thresh_by_method_list = []

for pre_process_save_dir in ['Preprocess_Files_v0_0/', 'Preprocess_Files_v0_4/']:
    pre_process_save_dir = out_dir + pre_process_save_dir
    
    area_percent_df = detect.get_thresh_outcome_time_series_dfs(
        HE_DATE_LIST, percent_of_peak_list, ALL_HE_DIR, pre_process_save_dir
    )[1]
    area_percent_df_by_method_list.append(area_percent_df)
    mad_by_thresh_by_method_list.append(
        detect.get_mad_by_confidences(area_percent_df, percent_of_peak_list)
    )

In [ ]:
x_ticks = np.arange(len(percent_of_peak_list))
threshold_label_list = [
    f'{thresh_level}% of Peak Threshold'
    for thresh_level in percent_of_peak_list
]

plt.figure(1, figsize=(8,6))

plt.bar(x_ticks - 0.2, mad_by_thresh_by_method_list[0], width=0.2, label='v0.3')
plt.bar(x_ticks, mad_by_thresh_by_method_list[1], width=0.2, label='Band Pass')
plt.bar(x_ticks + 0.2, mad_by_thresh_by_method_list[2], width=0.2, label='Rescaling')
plt.xticks(x_ticks, threshold_label_list, rotation=10)
plt.ylabel(f'MAD of Detected Area Percentage (%)')
plt.legend()

#### Preparation

Compare outcomes between confidence levels and/or methods

In [ ]:
out_dir = DETECT_DIR + '_Outcome_Comparison/' + DATE_DIR
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

confidence_level_list = [0, 35, 65, 95]
# confidence_level_list = list(range(0,96,5))

# version_dirs = ['v0_3/', 'Band_Pass/', 'Rescale/', 'Rescale_Center/']
# version_dirs = ['v0_3/', 'Rescale/']
# version_dirs = ['v0_3/', 'Rescale/', 'v0_4/']
# version_dirs = ['v0_3/', 'v0_4/']
# version_dirs = ['v0_4_Single/', 'v0_4/']
# version_dirs = ['v0_4_Unipolar']
version_dirs = ['v0_1', 'v0_2', 'v0_3', 'v0_4', 'v0_5']
descript_list = version_dirs + [f'cl{cl}' for cl in confidence_level_list]

Compute Outcomes

In [ ]:
area_percent_df_by_method_list = []
autocorr_by_conf_by_method_list = []
mad_by_conf_by_method_list = []
norm_mad_by_conf_by_method_list = []


for version_dir in version_dirs:
    ensemble_maps_save_dir = os.path.join(DETECT_DIR, version_dir, 'Saved_Map_Files/')
    
    area_percent_df, area_df = detect.get_outcome_time_series_dfs(
        HE_DATE_LIST, confidence_level_list, ensemble_maps_save_dir
    )[1:3]
    area_percent_df_by_method_list.append(area_percent_df)
    
    autocorr_by_confidences = [
        area_df[cl].autocorr()
        for cl in confidence_level_list
    ]
    autocorr_by_conf_by_method_list.append(autocorr_by_confidences)
    out = detect.get_mad_by_confidences(
        area_df, confidence_level_list
    )
    mad_by_confidences, norm_mad_by_confidences = out
    mad_by_conf_by_method_list.append(mad_by_confidences)
    norm_mad_by_conf_by_method_list.append(norm_mad_by_confidences)
    print(f'Outcomes computed for {version_dir}')

descript_list = version_dirs + [f'cl{cl}' for cl in confidence_level_list]
autocorr_file = f'{out_dir}Autocorr_comp_{"_".join(descript_list)}.npy'
np.save(autocorr_file, np.array(autocorr_by_conf_by_method_list),
        allow_pickle=True)

Plot Formatting

In [ ]:
cl_dx_list = np.arange(-0.3,0.31,0.2)
method_list = ['Bright & Coherent Mask', 'Ensemble', 'Smoothness',
               'Consistency', 'Unipolarity']

# cl_dx_list = np.arange(-0.9,0.91,0.2)
# method_list = ['Unipolarity']

# cl_dx_list = np.arange(0,1,0.05)
# method_list = ['Unipolarity']

# cl_dx_list = np.arange(-0.3,0.31,0.2)
# # method_list = ['v0.3', 'v0.3 Design + Band Pass', 'v0.3 Design + Rescale',
# #               'v0.3 Design + Rescale & Center']
# method_list = ['v0.1', 'v0.2', 'v0.3', 'v0.4']

# cl_dx_list = [-0.1, 0.1]
# # method_list = ['v0.3', 'v0.3 Design + Rescale']
# # method_list = ['v0.3', 'v0.4']
# method_list = ['v0.4 Single', 'v0.4 Ensemble']

# cl_dx_list = [-0.2, 0, 0.2]
# method_list = ['v0.3', 'v0.3 Design + Rescale', 'v0.4']

cmap = colormaps['viridis']
color_list = cmap(np.linspace(0, 0.75, len(confidence_level_list)))
# cmap = colormaps['plasma_r']
# color_list = cmap(np.linspace(0.25, 1, len(confidence_level_list)))

#### Design Variable Sweep

Area Sweep

In [ ]:
# area_file_name = f'{out_dir}Area_comp_{"_".join(descript_list)}'
area_percent_df = area_percent_df_by_method_list[0]
median_area_percent_by_cl = [
    np.median(area_percent_df[cl]) for cl in confidence_level_list
]

x_ticks = np.arange(len(method_list))
plt.figure(1, figsize=(9,6))

# Loop over confidence levels to plot bars for all methods at once
for median_area_percent, cl_dx, color in zip(
    median_area_percent_by_cl, cl_dx_list, color_list):
    plt.bar(x_ticks + cl_dx, median_area_percent, width=0.05, color=color)

plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Design Variable Sweep')
plt.ylabel('Median Detected Area Percentage (%)')
plt.xlabel('Unipolarity Threshold')
plt.xlim([-0.025,1.025])

Autocorrelation Sweep

In [ ]:
autocorr_file_name = f'{out_dir}Autocorr_comp_{"_".join(descript_list)}'
autocorrs_by_cl_by_method = np.load(autocorr_file_name + '.npy', allow_pickle=True)
autocorrs_by_method_by_cl = autocorrs_by_cl_by_method.T

x_ticks = np.arange(len(method_list))

plt.figure(1, figsize=(9,6))

# Loop over confidence levels to plot bars for all methods at once
for autocorrs_by_method, cl_dx, color in zip(
    autocorrs_by_method_by_cl, cl_dx_list, color_list):
    plt.bar(x_ticks + cl_dx, autocorrs_by_method, width=0.05,
            color=color)

plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Design Variable Sweep')
plt.ylabel(f'Autocorrelation')
plt.xlabel('Unipolarity Threshold')
plt.xlim([-0.025,1.025])

#### Method Comparison

Autocorrelation by Method

In [ ]:
autocorr_file_name = f'{out_dir}Autocorr_comp_{"_".join(descript_list)}'
autocorrs_by_cl_by_method = np.load(autocorr_file_name + '.npy', allow_pickle=True)
autocorrs_by_method_by_cl = autocorrs_by_cl_by_method.T

x_ticks = np.arange(len(method_list))
confidence_label_list = [
    f'{confidence_level}% Confidence'
    for confidence_level in confidence_level_list
]

plt.figure(1, figsize=(9,6))

# Loop over confidence levels to plot bars for all methods at once
for autocorrs_by_method, cl_dx, confidence, color in zip(
    autocorrs_by_method_by_cl, cl_dx_list, confidence_label_list, color_list):
    plt.bar(x_ticks + cl_dx, autocorrs_by_method, width=0.2,
            label=confidence, color=color)

plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Method Comparison')
plt.xticks(x_ticks, method_list)
plt.ylabel(f'Autocorrelation')

# plt.ylim([0, 0.8])
# plt.ylim([-0.1, 0.8])
# plt.axhline(0, color='k', linestyle='--')
plt.legend(loc='upper left')

# Save plot
plt.savefig(autocorr_file_name + '.png')
plt.close()
print(f'{autocorr_file_name.split("/")[-1]} method comparison saved.')

MAD

In [ ]:
x_ticks = np.arange(len(confidence_level_list))
confidence_label_list = [
    f'{confidence_level}% Confidence'
    for confidence_level in confidence_level_list
]

plt.figure(1, figsize=(9,6))
for mad_by_confidences, cl_dx, method, color in zip(
    mad_by_conf_by_method_list, cl_dx_list, method_list, color_list):
    plt.bar(x_ticks + cl_dx, mad_by_confidences, width=0.2,
            label=method, color=color)

plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Method Comparison')
plt.xticks(x_ticks, confidence_label_list, rotation=10)
plt.ylabel(f'MAD of Detected Area (Mm^2)')
plt.legend()

Normalized MAD

In [ ]:
x_ticks = np.arange(len(confidence_level_list))
confidence_label_list = [
    f'{confidence_level}% Confidence'
    for confidence_level in confidence_level_list
]

plt.figure(1, figsize=(9,6))
for norm_mad_by_confidences, cl_dx, method, color in zip(
    norm_mad_by_conf_by_method_list, cl_dx_list, method_list, color_list):
    plt.bar(x_ticks + cl_dx, norm_mad_by_confidences, width=0.2,
            label=method, color=color)
    
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Method Comparison')
plt.xticks(x_ticks, confidence_label_list, rotation=10)
plt.ylim([0, 50])
plt.ylabel(f'Normalized MAD of Detected Area (%)')
plt.legend()

## Write Images to Video

In [ ]:
detect.write_video(out_dir, fps=2)